In [1]:
import causal_nf.utils.wandb_local as wandb_local
import causal_nf.config as causal_nf_config
from causal_nf.config import cfg
import causal_nf.utils.training as causal_nf_train
import os
import glob
import argparse
from yacs.config import CfgNode 
import torch
import causal_nf.utils.io as causal_nf_io
import time
import numpy as np
from causal_nf.preparators.FF_preparator import FFPreparator
from causal_nf.config import cfg
from tqdm import tqdm

In [2]:
file_dir = './output_FF_ate5/c2d2c83c52bd11efa82ed90a43d0f393'
ckpt_file= f"{file_dir}/last.ckpt"

In [3]:
args_list = []
args =  CfgNode({'config_file': f'{file_dir}/wandb_local/config_local.yaml', 
                 'config_default_file': f'{file_dir}/wandb_local/default_config.yaml', 
                 'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                 'load_model': f'{file_dir}', 'delete_ckpt': False})
config = causal_nf_config.build_config(
    config_file=args.config_file,
    args_list=args_list,
    config_default_file=args.config_default_file,
)
causal_nf_config.assert_cfg_and_config(cfg, config)

In [4]:
preparator = FFPreparator.loader(cfg.dataset)
preparator.prepare_data()


Preparing data...

Preparing data...

Preparing data...


In [5]:
# IMPORTANT: insert ckpt file otherwise it loads anyway bt something wrong
model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=ckpt_file)
_ = model_lightning.model.eval()

NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from ./output_FF_ate5/c2d2c83c52bd11efa82ed90a43d0f393/last.ckpt
scaler_transform StandardTransform(shift=tensor([2.7234, 2.7408, 2.7082, 2.6903, 0.9523, 6.7677]), scale=tensor([2.7295, 2.6963, 2.7701, 2.7185, 0.2133, 1.5822]))
StandardScaler(
		mu_=tensor([[2.7234, 2.7408, 2.7082, 2.6903, 0.9523, 6.76

In [6]:
index = 4
a = 1
b = 0


ates = []
for seed in tqdm(range(25)):
    torch.random.manual_seed(seed + 1)
    ate = model_lightning.model.compute_ate(
        index,
        a=a,
        b=b,
        num_samples=10000,
        scaler=preparator.scaler_transform,
    )
    ates.append(ate[-1])
ates = np.array(ates)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:15<00:00,  1.58it/s]


In [7]:
ate_mean = ates.mean()
ate_stddev = ates.std()
print("Mean ATE = ", ate_mean)
print("Stddev ATE = ", ate_stddev)

Mean ATE =  4.2451477
Stddev ATE =  0.019233035
